# Suspect of too low accuracy in Lesion model
Vocabulary: 
- boardcasting: when adding tensor with different dimension, for example input_p (dim = [batch_size, 250]) and bias_p (dim = [1, 250]), bias_p will automatically "boardcast" to (batch_size, 250) during the elementwise addition

When input information have not reach a layer, it will assume the first axis (batch_size) = 1 
however, the batch_size won't matter... since tf.matmul batch_axis is independent with each other...  


In [ ]:
import tensorflow as tf
import pandas as pd
import altair as alt
import os, random
import meta, modeling, data_wrangling, evaluate, troubleshooting
from importlib import reload


# Parameter block

In [ ]:
code_name = 'Refrac3_local'
selected_word = 'pans'
epoch = 300
testset_name = 'train_r100'

## Instantiate troubleshooting class

In [ ]:
reload(troubleshooting)
d = troubleshooting.Diagnosis(code_name)

# Intact SEM

In [ ]:
d.eval(testset_name, task='triangle', epoch=epoch)
d.set_target_word(selected_word)
intact_sem = d.plot_one_layer('sem')
intact_sem.save(os.path.join(d.cfg.plot_folder, f"intact_sem_{selected_word}.html"))
intact_sem

# OS SEM

In [ ]:
d.eval(testset_name, task='ort_sem', epoch=epoch)
d.set_target_word(selected_word)
OS_sem = d.plot_one_layer('sem')
OS_sem.save(os.path.join(d.cfg.plot_folder, f"OS_sem_{selected_word}.html"))
OS_sem

# OPS SEM

In [ ]:
d.eval(testset_name, task='exp_ops', epoch=epoch)
d.set_target_word(selected_word)
OPS_sem = d.plot_one_layer('sem')
OPS_sem.save(os.path.join(d.cfg.plot_folder, f"OPS_sem_{selected_word}.html"))
OPS_sem

# Intact PHO

In [ ]:
d.eval(testset_name, task='triangle', epoch=epoch)
d.set_target_word(selected_word)
print(f"Output phoneme over timeticks: {d.get_output_phoneme()}")
intact_pho = d.plot_one_layer('pho')
intact_pho.save(os.path.join(d.cfg.plot_folder, f"intact_pho_{selected_word}.html"))
intact_pho


# OP PHO

In [ ]:
d.eval(testset_name, task='ort_pho', epoch=epoch)
d.set_target_word(selected_word)
print(f"Output phoneme over timeticks: {d.get_output_phoneme()}")
OP_pho = d.plot_one_layer('pho')
OP_pho.save(os.path.join(d.cfg.plot_folder, f"OP_pho_{selected_word}.html"))
OP_pho


# OSP PHO

In [ ]:
d.eval(testset_name, task='exp_osp', epoch=epoch)
d.set_target_word(selected_word)
print(f"Output phoneme over timeticks: {d.get_output_phoneme()}")
OSP_pho = d.plot_one_layer('pho')
OSP_pho.save(os.path.join(d.cfg.plot_folder, f"OSP_pho_{selected_word}.html"))
OSP_pho
